In [ ]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import ManagedOnlineDeployment, CodeConfiguration, Environment, Model
from azure.identity import DefaultAzureCredential

In [ ]:
!pip install docker

In [ ]:

credential = DefaultAzureCredential()
ml_client = MLClient(
    credential,
    subscription_id='49b5441f-dda4-47a9-81c4-13272430f4ff', 
    resource_group_name='aml-dbx-interconnect',
    workspace_name='amlws0001', 
)

In [ ]:
model = ml_client.models.get('nyctlc',version='1')
env='dev'
if env=='dev':
    Model(path='./deployment/model')

In [ ]:
deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name='pooyaalaviannyctlc',
    model=model,
    code_configuration=CodeConfiguration(
        code="./deployment", scoring_script="score.py"
    ),
    environment=Environment(
        conda_file="./deployment/conda.yaml",
        image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04",
    ),
    instance_type="Standard_DS3_v2",
    instance_count=1,
)

deployment = ml_client.online_deployments.begin_create_or_update(
    deployment, local=True, vscode_debug=True
).result()